In [1]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 8.6 MB/s eta 0:00:01    |███████████████▍                | 9.3 MB 8.6 MB/s eta 0:00:02
     |████████████████████████████████| 86 kB 5.2 MB/s  eta 0:00:01
     |████████████████████████████████| 457 kB 37.3 MB/s eta 0:00:01
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.3
    Uninstalling beautifulsoup4-4.9.3:
      Successfully uninstalled beautifulsoup4-4.9.3
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import json

from konlpy.tag import *

# 데이터 전처리

In [8]:
PAD = "<PADDING>"
STA = "<START>"
END = "<END>"
OOV = "<OOV>"

PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

max_sequences = 30
embedding_dim = 100
lstm_hidden_dim = 128

RE_FILTER = re.compile("[.,!?\"':;~()]")

In [4]:
chatbot_data = pd.read_csv('./processed_kakao.csv', encoding='utf-8')
questions, answers = list(chatbot_data['questions']), list(chatbot_data['answers'])

In [9]:
def pos_tag(sentences):
    tagger = Okt()
    sentences_pos = []
    for sentence in sentences:
        try:
            # 웃음 정규화
            sentence = re.sub('ㅋㅋㅋ+', 'ㅋㅋ', sentence)
            sentence = re.sub('ㅎㅎㅎ+', 'ㅎㅎ', sentence)
            # 특수기호 제거
            sentence = re.sub(RE_FILTER, "", sentence)
        except:
            sentence = "ㅇㅇ"
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence) 
    return sentences_pos

In [6]:
questions = pos_tag(questions)
answers = pos_tag(answers)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + questions[i])
    print('A : ' + answers[i])
    print()

Q : 나다
A : nada

Q : 방가 방가
A : ㅈㅈㅈ

Q : 어디서 봐야 돼
A : 꽁 짜로 보려면

Q : ㅇㅋㅇㅋ
A : 조선 니체 좌

Q : 청출어람 당할듯 ㅠ
A : @ 검 네이버 이 친구 도 알바로 집어넣어 버립시다

Q : 모르겠어
A : 그 또한 인생

Q : 갯수
A : 점점 늘어나는 잔고

Q : 다 들 니체 보고왔냐
A : 난

Q : ㅗ
A : 아 나

Q : 우어 아
A : 아우 어



In [7]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(questions)
sentences.extend(answers)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [8]:
# 단어 개수
len(words)

10072

In [9]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)} # 단어 -> 인덱스: 문장을 인덱스로 변환하여 모델 입력으로 사용
index_to_word = {index: word for index, word in enumerate(words)} # 인덱스 -> 단어: 모델의 예측 결과 인덱스를 문장으로 변환시 사용

In [10]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [11]:
x_encoder = convert_text_to_index(questions, word_to_index, ENCODER_INPUT) # 인코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answers, word_to_index, DECODER_INPUT) # 디코더 입력 인덱스 변환
y_decoder = convert_text_to_index(answers, word_to_index, DECODER_TARGET) # 디코더 목표 인덱스 변환

In [12]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

# 훈련모델 정의

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

In [25]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 예측모델 정의

In [26]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)



#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

# 훈련 및 테스트

In [39]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [43]:
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도/손실 출력
    print('accuracy :', history.history['accuracy'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    input_encoder = x_encoder[13].reshape(1, x_encoder[13].shape[0])
    input_decoder = x_decoder[13].reshape(1, x_decoder[13].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch : 1
accuracy : 0.9735598564147949
loss : 0.16587252914905548
검고 진짜 실망 이다 

Total Epoch : 2
accuracy : 0.9723795056343079
loss : 0.17532703280448914
검고 진짜 실망 이다 

Total Epoch : 3
accuracy : 0.9724055528640747
loss : 0.172832190990448
검고 진짜 실망 이다 

Total Epoch : 4
accuracy : 0.9733406901359558
loss : 0.16764171421527863
검고 진짜 실망 이다 

Total Epoch : 5
accuracy : 0.9742129445075989
loss : 0.16307130455970764
검고 진짜 실망 이다 

Total Epoch : 6
accuracy : 0.974395215511322
loss : 0.16149669885635376
검고 진짜 실망 이다 

Total Epoch : 7
accuracy : 0.9749983549118042
loss : 0.15853601694107056
검고 진짜 실망 이다 

Total Epoch : 8
accuracy : 0.9752479195594788
loss : 0.15832889080047607
검고 진짜 실망 이다 

Total Epoch : 9
accuracy : 0.9751502275466919
loss : 0.15715482831001282
검고 진짜 실망 이다 

Total Epoch : 10


KeyboardInterrupt: 

In [44]:
input_decoder = x_decoder[13].reshape(1, x_decoder[13].shape[0])
results = model.predict([input_encoder, input_decoder])

In [45]:
indexs = np.argmax(results[0], 1)

In [42]:
sentence = convert_index_to_text(indexs, index_to_word)
print(sentence)

검고 진짜 실망 이다 


# Output model

In [57]:
#save model
from tensorflow.python.keras.models import load_model
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, './model')
tfjs.converters.save_keras_model(encoder_model, './encoder_model')
tfjs.converters.save_keras_model(decoder_model, './decoder_model')

# SAVE json files
with open("word_to_index.json", "w") as json_file:
    json.dump(word_to_index_json, json_file)
with open("index_to_word.json", "w") as json_file:
    json.dump(index_to_word_json, json_file)

In [2]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

# 모델 테스트

In [3]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [10]:
while True:
    sen = input('[나] ')
    if sen == '그만': break
    input_seq = make_predict_input(sen)
    print(input_seq)
    output_seq = generate_text(input_seq)
    print('[혜성] {}'.format(output_seq))

[나] 혜성아안녕


NameError: name 'convert_text_to_index' is not defined